In [1]:
#open spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("A2-Part2-Pipeline").getOrCreate()

#load reviews and stopwords
DEBUG = True
if DEBUG:
    RAW_PATH = "hdfs:///user/dic25_shared/amazon-reviews/full/reviews_devset.json"
else:
    RAW_PATH = "hdfs:///user/dic25_shared/amazon-reviews/full/reviewscombined.json"
stopwordsPath = "Exercise_1/assets/stopwords.txt"
#define structure of json for faster reading
from pyspark.sql import types as T
review_schema = T.StructType([
     T.StructField("reviewerID",      T.StringType(),  True),
     T.StructField("asin",            T.StringType(),  True),
     T.StructField("reviewerName",    T.StringType(),  True),
     T.StructField("helpful",         T.ArrayType(T.IntegerType()), True),
     T.StructField("reviewText",      T.StringType(),  True),
     T.StructField("overall",         T.FloatType(),   True),
     T.StructField("summary",         T.StringType(),  True),
     T.StructField("unixReviewTime",  T.LongType(),    True),
     T.StructField("reviewTime",      T.StringType(),  True),
     T.StructField("category",        T.StringType(),  True),
 ])
#read and select category and review
df = (
    spark.read
         .schema(review_schema)
         .json(RAW_PATH)
         .selectExpr("reviewText AS text",
                     "category AS label")
    .na.drop(subset=["text", "label"])
)

# reading the stopwords
stopwords = spark.sparkContext.textFile(stopwordsPath).collect()


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/05/10 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/10 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/05/10 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/05/10 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/05/10 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/05/10 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
25/05/10 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
25/05/10 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
25/05/10 02:13:59 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [5]:
# ─────────────────────────────────────────────────────────────────────────────
#  Part 3 – Multiclass Text Classification with Linear-SVM  (PySpark)
# ─────────────────────────────────────────────────────────────────────────────
#
#  Prerequisite: the DataFrame `df` from Part 2
#                columns:  text(string) , label(string|int)
#  Spark ≥ 3.2.0 recommended
# ─────────────────────────────────────────────────────────────────────────────

from pyspark.ml import Pipeline
from pyspark.ml.feature import (
    RegexTokenizer, StopWordsRemover, CountVectorizer, IDF,
    Normalizer, ChiSqSelector, StringIndexer
)
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.sql import functions as F

# ─────────────────────────────────────────────────────────────────────────────
#  1.  Reproducible split: 80 % trainVal / 20 % test
#      … and inside the CV step below: another 80 % / 20 % split to get
#      train vs. *validation* (→ effective 64 % train · 16 % val · 20 % test)
# ─────────────────────────────────────────────────────────────────────────────
SEED = 567
train_val_df, test_df = df.randomSplit([0.8, 0.2], seed=SEED)
print(f"Train+Val: {train_val_df.count():,}  ·  Test: {test_df.count():,}")

# ─────────────────────────────────────────────────────────────────────────────
#  2.  String-labels → numeric (required for SVM)
# ─────────────────────────────────────────────────────────────────────────────
label_indexer = StringIndexer(inputCol="label", outputCol="label_idx").fit(df)

# ─────────────────────────────────────────────────────────────────────────────
#  3.  Shared preprocessing stages     (exactly as in Part 2)
# ─────────────────────────────────────────────────────────────────────────────
tokenizer = RegexTokenizer(
    inputCol="text", outputCol="tokens",
    pattern=r"""[ \t0-9()\[\]{}.!?,;:+=\-_"'`~#@&*%€$§\\/]+""",
    gaps=True, toLowercase=True)

stopper = StopWordsRemover(inputCol="tokens", outputCol="clean_tokens")

cv = CountVectorizer(
    inputCol="clean_tokens", outputCol="tf",
    minDF=2, vocabSize=50_000)        # keep generous ceiling

idf = IDF(inputCol="tf", outputCol="tfidf")

#  Normalise TF-IDF vectors
normaliser = Normalizer(inputCol="tfidf", outputCol="features", p=2.0)

#  Two χ² selectors (2 000 vs. 500 features)
selector2k = ChiSqSelector(
    numTopFeatures=2_000,
    featuresCol="features", outputCol="chi2_features",
    labelCol="label_idx")

selector500 = selector2k.copy({selector2k.numTopFeatures: 500})

#  SVM wrapped for multi-class
svm = LinearSVC(featuresCol="chi2_features",
                labelCol="label_idx",
                predictionCol="prediction")

ovr = OneVsRest(classifier=svm,
                labelCol="label_idx",
                featuresCol="chi2_features",
                predictionCol="prediction")

# ─────────────────────────────────────────────────────────────────────────────
#  4.  Pipeline (place-holders; grid will swap in selector2k vs. selector500)
# ─────────────────────────────────────────────────────────────────────────────
pipe = Pipeline(stages=[
    label_indexer,         # [0]
    tokenizer, stopper, cv, idf, normaliser,
    selector2k,            # [6] will be overridden by grid
    ovr                    # [7]
])

# ─────────────────────────────────────────────────────────────────────────────
#  5.  Param grid
#      • χ²: 2 000  vs  500
#      • regParam:  0.01, 0.1, 1.0
#      • standardization:  True / False
#      • maxIter:  50 / 200
# ─────────────────────────────────────────────────────────────────────────────
param_grid = (
    ParamGridBuilder()
    # swap χ²-selector stage
    #.addGrid(pipe.stages[6], selector2k)
    # SVM hyper-params (access the underlying LinearSVC inside the OvR)
    .addGrid(svm.regParam,        [0.01, 0.1, 1.0])
    .addGrid(svm.standardization, [True, False])
    .addGrid(svm.maxIter,         [50, 200])
    .build()
)

print(f"Grid size: {len(param_grid):,} parameter combos")

# ─────────────────────────────────────────────────────────────────────────────
#  6.  Train-validation split inside **TrainValidationSplit**
#      (20 % of trainVal_df is held out for validation)
# ─────────────────────────────────────────────────────────────────────────────
evaluator = MulticlassClassificationEvaluator(
    labelCol="label_idx",
    predictionCol="prediction",
    metricName="f1")

tvs = TrainValidationSplit(
    estimator=pipe,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    trainRatio=0.8,           # 80 % train, 20 % val (of trainVal_df)
    seed=SEED,
    parallelism=4)            # adjust to cluster resources

# ─────────────────────────────────────────────────────────────────────────────
#  7.  Fit grid & choose best model
# ─────────────────────────────────────────────────────────────────────────────
tvs_model = tvs.fit(train_val_df)
best_model = tvs_model.bestModel
best_params = {k.name: v for k, v in best_model.stages[-1]  # OvR → LinearSVCModel
                                       .getClassifier()
                                       .extractParamMap().items()}
print("Best params:", best_params)

# ─────────────────────────────────────────────────────────────────────────────
#  8.  Final evaluation on the untouched test set
# ─────────────────────────────────────────────────────────────────────────────
pred_test = best_model.transform(test_df)
f1_test = evaluator.evaluate(pred_test)
print(f"Test F1 (macro): {f1_test:.4f}")


Train+Val: 63,040  ·  Test: 15,789
Grid size: 12 parameter combos


25/05/10 02:25:09 WARN DAGScheduler: Broadcasting large task binary with size 1034.6 KiB


25/05/10 02:25:14 WARN DAGScheduler: Broadcasting large task binary with size 1034.6 KiB


25/05/10 02:25:20 WARN DAGScheduler: Broadcasting large task binary with size 1034.6 KiB


25/05/10 02:25:20 WARN DAGScheduler: Broadcasting large task binary with size 1036.7 KiB


...

25/05/10 02:30:46 WARN TaskSetManager: Lost task 0.3 in stage 143.0 (TID 284) (lbdwo17.datalab.novalocal executor 1): org.apache.spark.SparkException: Chi-square test expect factors (categorical values) but found more than 10000 distinct values in column 2.
	at org.apache.spark.mllib.stat.test.ChiSqTest$.computeChiSq(ChiSqTest.scala:195)
	at org.apache.spark.mllib.stat.test.ChiSqTest$.$anonfun$chiSquaredSparseFeatures$15(ChiSqTest.scala:175)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at scala.collection.Iterator$$anon$10.ha

[Stage 145:>                (0 + 1) / 2][Stage 146:>                (0 + 1) / 2]

25/05/10 02:30:48 WARN TaskSetManager: Lost task 1.0 in stage 145.0 (TID 287) (lbdwo17.datalab.novalocal executor 1): org.apache.spark.SparkException: Chi-square test expect factors (categorical values) but found more than 10000 distinct values in column 3.
	at org.apache.spark.mllib.stat.test.ChiSqTest$.computeChiSq(ChiSqTest.scala:195)
	at org.apache.spark.mllib.stat.test.ChiSqTest$.$anonfun$chiSquaredSparseFeatures$15(ChiSqTest.scala:175)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at scala.collection.Iterator$$anon$10.ha

[Stage 147:>                                                        (0 + 2) / 2]

25/05/10 02:31:03 WARN TaskSetManager: Lost task 1.0 in stage 147.0 (TID 293) (lbdwo17.datalab.novalocal executor 1): org.apache.spark.SparkException: Chi-square test expect factors (categorical values) but found more than 10000 distinct values in column 3.
	at org.apache.spark.mllib.stat.test.ChiSqTest$.computeChiSq(ChiSqTest.scala:195)
	at org.apache.spark.mllib.stat.test.ChiSqTest$.$anonfun$chiSquaredSparseFeatures$15(ChiSqTest.scala:175)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at scala.collection.Iterator$$anon$10.ha